# 02 — Bond Pricing Engine (Clean / Dirty / Accrued)

## Objective
Build a pricing workflow for fixed-rate coupon bonds that returns:
- **Dirty price**: PV of remaining cashflows (includes accrued)
- **Accrued interest** $AI$
- **Clean price**: $P_{clean} = P_{dirty} - AI$

## Settlement-adjusted discounting
We value at settlement date $s$ (not valuation timestamp $t_0$) by using:
$$
D_s(T) = \frac{D(T)}{D(s)}.
$$

We discount cashflows from settlement, not from the curve’s anchor date.

## Day count impact
Accrued interest depends strongly on day count convention (e.g., 30/360 vs ACT/365).
We treat this as *instrument-level* convention and keep the curve day count separate.

In [1]:
import numpy as np
import pandas as pd

from fixed_income_engine.curves import bootstrap_curve_from_bills_notes
from fixed_income_engine.bonds import (
    Bond,
    BondPricer,
    settlement_date,
)

# Build curve (repeatable notebook)
val_date = pd.Timestamp("2026-02-13")
market = pd.DataFrame([
    {"type": "bill", "maturity": pd.Timestamp("2026-02-20"), "quote": 0.0525, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-03-13"), "quote": 0.0520, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-05-15"), "quote": 0.0515, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-08-14"), "quote": 0.0505, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2027-02-12"), "quote": 0.0485, "day_count": "ACT/360"},
    {"type": "note", "maturity": pd.Timestamp("2028-02-15"), "quote": 0.0450, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2031-02-15"), "quote": 0.0430, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2036-02-15"), "quote": 0.0425, "coupon_freq": 2, "day_count": "30/360"},
]).sort_values("maturity").reset_index(drop=True)

curve = bootstrap_curve_from_bills_notes(market, val_date)
pricer = BondPricer(curve)

In [2]:
bond = Bond(
    bond_id="CORP_5Y_5PCT",
    maturity=pd.Timestamp("2031-02-15"),
    coupon_rate=0.05,
    freq=2,
    day_count="30/360",
    face=100.0,
)

settle = settlement_date(val_date, lag_days=2)
dirty, clean, ai = pricer.price(bond, val_date, settle)

print("Settlement:", settle.date())
print("Dirty:", dirty)
print("Accrued:", ai)
print("Clean:", clean)

Settlement: 2026-02-15
Dirty: 100.93844252553204
Accrued: 0.0
Clean: 100.93844252553204


In [3]:
# Accrued behavior around coupon date
test_settles = [pd.Timestamp("2026-02-14"), pd.Timestamp("2026-02-15"), pd.Timestamp("2026-02-16")]

for s in test_settles:
    dirty, clean, ai = pricer.price(bond, val_date, s)
    print(s.date(), "AI:", ai, "Dirty:", dirty, "Clean:", clean)

2026-02-14 AI: 2.486111111111111 Dirty: 103.42336654570346 Clean: 100.93725543459234
2026-02-15 AI: 0.0 Dirty: 100.93844252553204 Clean: 100.93844252553204
2026-02-16 AI: 0.01388888888888889 Dirty: 100.95315627905092 Clean: 100.93926739016203
